This file will follow tutorial 
https://www.depends-on-the-definition.com/introduction-named-entity-recognition-python/

In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    file.close()
    return contents

def idividual_read_file( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence #" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

def read_file( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence #" : [] , "Word" : [] , "Tag" : [] } )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = idividual_read_file( word , # raw_data
                                              ["\n"] , # word show split sentence
                                              count , # order of sentence
                                              [ " " , '' , '\0'] ) # word to delete or prevent
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = idividual_read_file( word ,
                                              ["\n"] ,
                                              count ,
                                              [ " " , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame

In [3]:
# Variable for collect name file and access file
directory = "corpus_directory"
train_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" )
test_files = ("result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )

In [4]:
data = read_file( directory , train_files , 1 )
data.fillna( method="ffill")
data.tail( 10 )

,Sentence #,Word,Tag
52212,Sentence: 3762,แห่ง,org_cont
52213,Sentence: 3762,ชาติ,org_end
52214,Sentence: 3762,หรือ,O
52215,Sentence: 3762,ป.ป.ช.,org
52216,Sentence: 3762,ต้อง,O
52217,Sentence: 3762,ประกาศ,O
52218,Sentence: 3762,หลักเกณฑ์,O
52219,Sentence: 3762,และ,O
52220,Sentence: 3762,จำนวน,O
52221,Sentence: 3762,ทรัพย์สิน,O


In [5]:
def merge_all_tag( data ):
    for run in range(  0 , data.shape[0] ):
        if data.Tag[run] in ( '' ):
            data.Tag[run] = 'O'
        elif data.Tag[run] in ('loc_cont','loc_end','loc_start'):
            data.Tag[run] = 'loc'
        elif data.Tag[run] in ('org_cont','org_end','org_start'):
            data.Tag[run] = 'org'
        elif data.Tag[run] in ('per_cont','per_end','per_start'):
            data.Tag[run] = 'per'
        else:
            None

In [6]:
set( data.Tag)

{'',
 'O',
 'loc',
 'loc_cont',
 'loc_end',
 'loc_start',
 'org',
 'org_cont',
 'org_end',
 'org_start',
 'per',
 'per_cont',
 'per_end',
 'per_start'}

In [7]:
merge_all_tag( data ) 

In [8]:
set( data.Tag )

{'O', 'loc', 'org', 'per'}

In [9]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
        try:
            s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            print(s)
            return s["Word"].values.tolist(), s["Tag"].values.tolist()   
        except:
            self.empty = True
            return None, None

In [10]:
getter = SentenceGetter( data )

In [11]:
sent, tag = getter.get_next()

     Sentence #        Word  Tag
0   Sentence: 1           "    O
1   Sentence: 1  พลอากาศเอก    O
2   Sentence: 1      ทักษิณ  per
3   Sentence: 1           "    O
4   Sentence: 1          ขอ    O
5   Sentence: 1       โอกาส    O
6   Sentence: 1         ให้    O
7   Sentence: 1           "    O
8   Sentence: 1     พล.ต.ท.    O
9   Sentence: 1        เสรี  per
10  Sentence: 1           "    O


In [12]:
print(sent);print(tag)

['"', 'พลอากาศเอก', 'ทักษิณ', '"', 'ขอ', 'โอกาส', 'ให้', '"', 'พล.ต.ท.', 'เสรี', '"']
['O', 'O', 'per', 'O', 'O', 'O', 'O', 'O', 'O', 'per', 'O']


ข้างบน จะดึงออกมาทีละประโยค หมายความว่าถ้าคุณต้องการหมด คุณต้องเรียกไปเรื่อย ๆ เลย

In [13]:
from sklearn.base import BaseEstimator , TransformerMixin

In [14]:
class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {}
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [15]:
tagger = MemoryTagger()

In [16]:
tagger.fit( sent , tag )

In [17]:
print( tagger.predict( sent ) )

['O', 'O', 'per', 'O', 'O', 'O', 'O', 'O', 'O', 'per', 'O']


ข้างบน จะเริ่มทำการเก็บข้อมูล ในรูปแบบของหน่วยความจำ ถ้ามีคำนี้จะจำว่าโอเคใช้ name entity นี้นะ ถ้าไม่มีในหน่วยความจำจะใช้วิธีให้ว่าเป็น 'O' แทน

In [19]:
# ref for first module https://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

In [23]:
test_words , test_tags = getter.get_next()

     Sentence #          Word  Tag
64  Sentence: 3        พ.ต.ท.    O
65  Sentence: 3        ทักษิณ  per
66  Sentence: 3                per
67  Sentence: 3       ชินวัตร  per
68  Sentence: 3  นายกรัฐมนตรี    O
69  Sentence: 3            ใน    O
70  Sentence: 3          ฐานะ    O
71  Sentence: 3       หัวหน้า    O
72  Sentence: 3          พรรค    O
73  Sentence: 3     ไทยรักไทย  org
74  Sentence: 3         กล่าว    O
75  Sentence: 3         เมื่อ    O
76  Sentence: 3        วันที่    O
77  Sentence: 3             7    O
78  Sentence: 3          พ.ย.    O
79  Sentence: 3     ที่ผ่านมา    O
80  Sentence: 3           ว่า    O
81  Sentence: 3           ไม่    O
82  Sentence: 3           น่า    O
83  Sentence: 3            มี    O
84  Sentence: 3          เป็น    O
85  Sentence: 3        เรื่อง    O
86  Sentence: 3          ความ    O
87  Sentence: 3      เป็นกลาง    O


In [24]:
pred = cross_val_predict(estimator=MemoryTagger(), X=test_words, y=test_tags, cv=5)

In [26]:
report = classification_report(y_pred=pred, y_true=test_tags)
print( report )

              precision    recall  f1-score   support

           O       0.83      1.00      0.91        20
         org       0.00      0.00      0.00         1
         per       0.00      0.00      0.00         3

    accuracy                           0.83        24
   macro avg       0.28      0.33      0.30        24
weighted avg       0.69      0.83      0.76        24

